In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [3]:
labels = df[1].values

In [4]:
type(labels)

numpy.ndarray

In [5]:
texts = df[0].values.tolist()

In [6]:
from transformers import ElectraTokenizer, TFElectraForSequenceClassification

In [7]:
tokenizer = ElectraTokenizer.from_pretrained("google/electra-base-discriminator")

In [8]:
from tensorflow.keras.utils import to_categorical
y_one_hot = to_categorical(labels)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(texts, y_one_hot, test_size=0.2, random_state=0)

In [10]:
X_train_tokenized = tokenizer(X_train, return_tensors="np", max_length=30, padding='max_length', truncation=True)
X_test_tokenized = tokenizer(X_test, return_tensors="np", max_length=30, padding='max_length', truncation=True)

In [11]:
model = TFElectraForSequenceClassification.from_pretrained("google/electra-base-discriminator", num_labels=2)

Some layers from the model checkpoint at google/electra-base-discriminator were not used when initializing TFElectraForSequenceClassification: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
optimizer = tf.keras.optimizers.Adam(2e-5)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [13]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
checkpoint_filepath = "./checkpoints/checkpoint_electra_en"
mc = ModelCheckpoint(checkpoint_filepath, monitor='val_loss', mode='min', 
                     save_best_only=True, save_weights_only=True)

In [14]:
model.fit(dict(X_train_tokenized), y_train, epochs=10, batch_size=128, 
          validation_split=0.1, callbacks=[es, mc])

Epoch 1/10
39/39 [==============================] - 920s 23s/step - loss: 0.6019 - accuracy: 0.7294 - val_loss: 0.3665 - val_accuracy: 0.9170
Epoch 2/10
39/39 [==============================] - 940s 24s/step - loss: 0.3008 - accuracy: 0.9113 - val_loss: 0.2304 - val_accuracy: 0.9278
Epoch 3/10
39/39 [==============================] - 865s 22s/step - loss: 0.1751 - accuracy: 0.9484 - val_loss: 0.2155 - val_accuracy: 0.9314
Epoch 4/10
39/39 [==============================] - 634s 16s/step - loss: 0.1119 - accuracy: 0.9697 - val_loss: 0.2402 - val_accuracy: 0.9242
Epoch 5/10
39/39 [==============================] - 596s 15s/step - loss: 0.0824 - accuracy: 0.9777 - val_loss: 0.2398 - val_accuracy: 0.9296
Epoch 6/10
39/39 [==============================] - 620s 16s/step - loss: 0.0631 - accuracy: 0.9833 - val_loss: 0.2835 - val_accuracy: 0.9242
Epoch 6: early stopping


In [15]:
model.load_weights(checkpoint_filepath)

In [16]:
model.evaluate(dict(X_test_tokenized), y_test)

44/44 [==============================] - 64s 1s/step - loss: 0.2133 - accuracy: 0.9306


[0.2132836878299713, 0.9306358098983765]

In [17]:
y_preds = model.predict(dict(X_test_tokenized))
prediction_probs = tf.nn.softmax(y_preds.logits,axis=1).numpy()
y_predictions = np.argmax(prediction_probs, axis=1)
y_test = np.argmax(y_test, axis=1)
from sklearn.metrics import classification_report
print(classification_report(y_predictions, y_test))

44/44 [==============================] - 70s 1s/step
              precision    recall  f1-score   support

           0       0.93      0.92      0.93       652
           1       0.93      0.94      0.93       732

    accuracy                           0.93      1384
   macro avg       0.93      0.93      0.93      1384
weighted avg       0.93      0.93      0.93      1384

